In [1]:
import os
import pandas as pd

def exploding( file_address ):
    df  = pd.read_csv(file_address)
    df['proposal_ids'] = df['proposal_ids'].map(lambda x: eval(x))
    df['author'] = df['author'].map(lambda x: eval(x))
    df['choice'] = df['choice'].map(lambda x: eval(x))
    df['pac'] = df.apply(lambda x:list(zip(*[x['proposal_ids'], x['author'], x['choice']])) , axis = 1)

    df_e = df.explode('pac').reset_index(drop = True)

    df_e['proposal_ids'] = df_e['pac'].map(lambda x: x[0])
    df_e['author'] = df_e['pac'].map(lambda x: x[1])
    df_e['choice'] = df_e['pac'].map(lambda x: x[2])
    
    return df_e

def find_reciprocal_relationship(df_e):
    df_c = df_e[(df_e['voter_address'].isin(df_e['author']) & df_e['voter_address'].isin(df_e['author']))] 

    df_c_condition1 = df_c[['voter_address','author']].apply(lambda x: str(sorted(x)), axis=1)
    df_c_condition2 = df_c[['voter_address','author']].apply(lambda x: str(sorted(x)), axis=1).value_counts()

    df_c2 = df_c[df_c_condition1.isin(df_c_condition2[df_c_condition2>1].index)].copy()

    df_c2['va_sorted'] = df_c2[['voter_address','author']].apply(lambda x: str(sorted(x)), axis=1)
    df_c2['same_as_sorted'] = df_c2['va_sorted'] == df_c2[['voter_address','author']].apply(lambda x: str([x['voter_address'],x['author']]),axis=1)

    reciprocal_ids = df_c2.groupby(['va_sorted'])['same_as_sorted'].unique()[df_c2.groupby(['va_sorted'])['same_as_sorted'].unique().map(len) > 1]
    
    relationship = df_c2[df_c2['va_sorted'].isin(reciprocal_ids.index)]

    return relationship
    


In [ ]:
# Define the directory path
directory_path = r"D:\DAO_Voting_1101\Statistics\voter_stats\multiple_times_voters"

# Output directory for reciprocal relationships
output_directory = r"D:\DAO_Voting_1101\Statistics\voter_stats\reciprocity"

# Iterate through each CSV file in the directory
for filename in sorted(os.listdir(directory_path))[]:
    
    if filename.endswith(".csv"):
        # Read the CSV file
        file_path = os.path.join(directory_path, filename)

        df_e = exploding(file_path)

        relationship = find_reciprocal_relationship(df_e)
        relationship_drop = relationship.drop(['unique_authors', 'num_unique_authors', 'pac', 'va_sorted', 'same_as_sorted'], axis=1)
        relationship_drop.to_csv(os.path.join(output_directory, f"{filename.split('.')[0].split('_')[0]}_reciprocal_relationships.csv"), index=False)

    print(f"Finished processing {filename}")